In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
sys.path.insert(1, "/shared/opencv/release/lib/python2.7/dist-packages/")
import cv2
import random
from collections import defaultdict

from sklearn.cluster import KMeans
import pandas as pd

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from sift_spm_utilities import *
from learning_utilities import *
from annotation_utilities import *

Setting environment for AWS compute node


No vtk
Cannot import mxnet.


In [2]:
# CSHL_SPM_ROOTDIR = '/shared/CSHL_SIFT_SPM_features/'
M = 200 # vocabulary size
colors = np.vstack([(0,0,0), np.random.randint(0, 255, (M, 3))])

In [3]:
win_id = 1

In [4]:
stack = 'MD589'

# Build Vocabulary

In [ ]:
for stack in ['MD589']:
    for sec in metadata_cache['valid_sections'][stack]:
        
        keypoints, descriptors = compute_sift_keypoints_and_descriptors(stack=stack, section=sec)
        
        fp = DataManager.get_sift_keypoints_filepath(stack=stack, section=sec)
        create_parent_dir_if_not_exists(fp)
        bp.pack_ndarray_file(keypoints, fp)
        upload_to_s3(fp)
        
        fp = DataManager.get_sift_descriptors_filepath(stack=stack, section=sec)
        create_parent_dir_if_not_exists(fp)
        bp.pack_ndarray_file(descriptors, fp)
        upload_to_s3(fp)

In [28]:
vocab_fp = DataManager.get_sift_descriptor_vocabulary_filepath()
download_from_s3(vocab_fp)
vocabulary_kmeans = joblib.load(vocab_fp)

In [8]:
# viz = gray2rgb(img)
# for x, y in keypoints:
#     cv2.circle(viz, (int(x), int(y)), 5, (255,0,0))

In [9]:
# display_image(viz[1000:5000, 1000:5000])

# or use author's own binary, 10x slower.

In [42]:
from skimage.io import load_sift

In [ ]:
import uuid

for stack in ['MD589']:
    for sec in metadata_cache['valid_sections'][stack][:1]:
        
        image = DataManager.load_image_v2(stack=stack, section=134, prep_id=2, version='gray')
        mask = DataManager.load_image_v2(stack=stack, section=134, prep_id=2, version='mask', resol='thumbnail')
        xmin_tb, xmax_tb, ymin_tb, ymax_tb = bbox_2d(mask)
        xmin = xmin_tb * 32
        xmax = (xmax_tb + 1) * 32
        ymin = ymin_tb * 32
        ymax = (ymax_tb + 1) * 32

        img = image[ymin:ymax+1, xmin:xmax+1].copy()
        
        keypoints_allBlocks = []
        descriptors_allBlocks = []
        
        # the binary cannot deal with large data
        for y in range(0, img.shape[0], 3000):
            for x in range(0, img.shape[1], 3000):
                t = time.time()
                
                uid = str(uuid.uuid1()).split('-')[0]
                in_fp = '/tmp/%s.pgm' % uid
                imsave(in_fp, img[y:y+3000, x:x+3000])
                out_fp = '/tmp/%s.key' % uid
                execute_command('/shared/siftDemoV4/sift <%s >%s' % (in_fp, out_fp))

                sift_data = load_sift(out_fp)
                descriptors = sift_data['data'].shape
                keypoints = np.c_[sift_data['column'], sift_data['row']]
                print len(keypoints), 'keypoints'
                # sift_data['orientation']
                # sift_data['scale']
                
                keypoints_allBlocks.append(keypoints)
                descriptors_allBlocks.append(descriptors)

        #         keypoints, descriptors = sift.detectAndCompute(img, None);
                sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) 
            
        keypoints = np.concatenate(keypoints_allBlocks)
        descriptors = np.concatenate(descriptors_allBlocks)

In [ ]:
# 

In [50]:
for stack in ['MD589']:
    for sec in metadata_cache['valid_sections'][stack]:
        labelmap = compute_labelmap(stack=stack, section=sec)
        labelmap_fp = DataManager.get_sift_descriptors_labelmap_filepath(stack, section=sec)
        create_parent_dir_if_not_exists(labelmap_fp)
        bp.pack_ndarray_file(labelmap, labelmap_fp)
        upload_to_s3(labelmap_fp)

aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_sift_labelmap.bp"
630146 keypoints


compute vocab: 0.00 seconds
predict: 1.23 seconds


aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-IHC16-2015.07.30-22.20.07_MD589_3_0048_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-IHC16-2015.07.30-22.20.07_MD589_3_0048_sift_labelmap.bp"
670222 keypoints


compute vocab: 0.00 seconds
predict: 1.27 seconds


aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-N17-2015.07.30-17.07.32_MD589_1_0049_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-N17-2015.07.30-17.07.32_MD589_1_0049_sift_labelmap.bp"
714307 keypoints


...

aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-N69-2015.07.30-20.46.59_MD589_2_0206_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-N69-2015.07.30-20.46.59_MD589_2_0206_sift_labelmap.bp"
627573 keypoints


compute vocab: 0.00 seconds
predict: 1.19 seconds


aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-IHC69-2015.07.31-02.07.15_MD589_2_0206_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-IHC69-2015.07.31-02.07.15_MD589_2_0206_sift_labelmap.bp"
618177 keypoints


compute vocab: 0.00 seconds
predict: 1.17 seconds


aws s3 cp "/shared/CSHL_SPM/sift_labelmap/MD589/MD589-N69-2015.07.30-20.46.59_MD589_3_0207_sift_labelmap.bp" "s3://mousebrainatlas-data/CSHL_SPM/sift_labelmap/MD589/MD589-N69-2015.07.30-20.46.59_MD589_3_0207_sift_labelmap.bp"


In [52]:
structure = '7N'

In [70]:
_, patch_locations = DataManager.load_patch_locations(stack=stack, win=win_id, section=sec)

In [ ]:
# for stack in ['MD589', 'MD594', 'MD585']:
for stack in ['MD589']:

    patch_size = windowing_settings[win_id]['patch_size']

    for sec in metadata_cache['valid_sections'][stack][60:61]:
        print sec
        labelmap = compute_labelmap(stack=stack, section=sec)
        hists_arr0, hists_arr1, hists_arr2 = compute_spm_histograms(labelmap, patch_locations, patch_size=patch_size, M=M)


153
